In [1]:
!where python

c:\Users\valen\Documents\Informatique-L3\Stage_NER\NER\.venv\Scripts\python.exe
C:\Users\valen\AppData\Local\Microsoft\WindowsApps\python.exe


In [1]:
from tools import casen, spacy_wrapper, ner
import importlib

In [2]:
EXCEL_DATA = "C:\\Users\\valen\\Documents\\Informatique-L3\\Stage_NER\\NER\\Ressources\\20231101_raw.xlsx"
EXCEL_CORRECTION = "C:\\Users\\valen\\Documents\\Informatique-L3\\Stage_NER\\NER\\NER\\Results\\NER.xlsx"
GRF = "C:\\Users\\valen\\Documents\\Informatique-L3\\Stage_NER\\NER\\NER\\optimisations\\grf.json"
EXCLUDED_NAMES = "C:\\Users\\valen\\Documents\\Informatique-L3\\Stage_NER\\NER\\NER\\optimisations\\name.json"
PIPELINE_RESULT = "C:\\Users\\valen\\Documents\\Informatique-L3\\Stage_NER\\NER\\NER\\Results"
CASEN = "C:\\Users\\valen\\Documents\\Informatique-L3\\Stage\\CasEN_fr.2.0\\CasEN.ipynb"
CASEN_CORPUS_FOLDER = "C:\\Users\\valen\\Documents\\Informatique-L3\\Stage_NER\\NER\\Results\\Corpus"
CASEN_RESULT_FOLDER = "C:\\Users\\valen\\Documents\\Informatique-L3\\Stage_NER\\NER\\Results\\CasEN\\Res_CasEN"
LOG_FOLDER = "C:\\Users\\valen\\Documents\\Informatique-L3\\Stage_NER\\NER\\NER\\Logs"
ARCHIVES_FOLDER = "C:\\Users\\valen\\Documents\\Informatique-L3\\Stage_NER\\NER\\NER\\Archives"

In [5]:
# importlib.reload(stanza_wrapper)
# from tools.stanza_wrapper import Stanza

# st = Stanza(
#     data= EXCEL_DATA,
#     use_gpu = False,
#     logging = True,
#     log_folder= LOG_FOLDER,
#     timer = True,
#     verbose = False
# )

# st_df = st.run()

In [6]:
#st_df.to_excel("20231101_stanza.xlsx")
# import pandas as pd
# st_df = pd.read_excel("./Results/20231101_stanza.xlsx")
# st_df.head()

In [9]:
importlib.reload(spacy_wrapper)
from tools.spacy_wrapper import SpaCy

s = SpaCy(
    data = EXCEL_DATA,
    spaCy_model = "fr_core_news_sm",
    timer_option = True,
    log_option = True,
    log_path = LOG_FOLDER,
    verbose = True
)
s_df = s.run()
#s_df.to_excel("new_spacy.xlsx")

[spaCy] spaCy version: 3.8.7
[spaCy] spaCy model: core_news_sm
run in : 83.26s


In [10]:
importlib.reload(casen)
from tools.casen import CasEN

c = CasEN(
    path= CASEN,
    corpus_folder = CASEN_CORPUS_FOLDER,
    result_folder = CASEN_RESULT_FOLDER,
    data = EXCEL_DATA,
    remove_MISC = True,
    archive_folder = None,
    single_corpus = True,
    run_casEN = False,
    logging = True,
    log_folder = LOG_FOLDER,
    timer = True,
    verbose = True
)
c_df = c.run()
#c_df.head()
# c_df.to_excel("new_casen.xlsx")

load_data in : 1.30s
[load] 1 file(s) loaded
get_entities in : 2.02s
CasEN in : 2.63s
run in : 2.63s


In [20]:
c_df.to_excel("new_casen.xlsx")

In [25]:
importlib.reload(ner)
from tools.ner import NER

N = NER(
    spaCy = s_df,
    casEN = c_df,
    data = EXCEL_DATA,
    casEN_priority_merge = True,
    casEN_graph_validation = GRF,
    extent_optimisation = False,
    remove_duplicate_rows = False,
    ner_result_folder = PIPELINE_RESULT,
    excluded_names = EXCLUDED_NAMES,
    correction = EXCEL_CORRECTION,
    logging = True,
    log_folder = LOG_FOLDER,
    timer = True,
    verbose = True
)
N.run()
N.df.head()

[merge] CasEN's DataFrame : (12437, 9)
[merge] SpaCy's DataFrame : (20912, 6)
[merge] Merge's DataFrame : (27535, 9)
[merge] SpaCy only      : 15098 lignes
[merge] CasEN only      : 6623 lignes
[merge] Intersection    : 5814 lignes
merge in : 0.45s
[casEN_optimisation] SpaCy only      : 15098 lignes
[casEN_optimisation] CasEN only      : 5908 lignes
[casEN_optimisation] CasEN_opti only      : 715 lignes
[casEN_optimisation] Intersection    : 5814 lignes
casEN_optimisation in : 0.80s
[casEN_priority] 2339 conflicting entities found (spaCy vs casEN)
[casEN_priority] 1662 added.
[casEN_priority] SpaCy only      : 15098 lignes
[casEN_priority] CasEN only      : 5908 lignes
[casEN_priority] CasEN_opti only      : 715 lignes
[casEN_priority] CasEN_priority only      : 1318 lignes
[casEN_priority] Intersection    : 5814 lignes
casEN_priority in : 0.12s
apply_correction in : 2.07s
save_dataframe in : 6.29s
run in : 11.36s


,manual cat,correct,extent,category,titles,NER,NER_label,desc,method,main_graph,second_graph,third_graph,file_id
0,PER,1.0,1.0,1,Faster than fear,Haffner,PER,lle n'a plus rien à voir avec l'affaire Haffne...,spaCy,NaN,NaN,NaN,0.0
1,PER,1.0,1.0,1,Faster than fear,Haffner,PER,"e s'adresser à... elle. En garde à vue, Haffne...",spaCy,NaN,NaN,NaN,0.0
2,NaN,NaN,NaN,NaN,Faster than fear,Nora,LOC,"nny. D'ailleurs, elle est persuadée que Nora a...",spaCy,NaN,NaN,NaN,0.0
3,NaN,NaN,NaN,NaN,Faster than fear,Sunny,PER,Ralf a pu prouver son innocence et Sunny a été...,spaCy,NaN,NaN,NaN,0.0
4,PER,0.0,1.0,PER,Faster than fear,Sunny,LOC,mais celui-ci demande à ne parler qu'à Sunny....,spaCy,NaN,NaN,NaN,0.0


In [ ]:
from tools import casen_parser
importlib.reload(casen_parser)
from tools.casen_parser import CasENParser


parser = CasENParser()

with open("C:\\Users\\valen\\Documents\\Informatique-L3\\Stage_NER\\NER\\Results\\CasEN\\Res_CasEN\\corpus.result.txt", 'r', encoding="utf-8") as f:
    text = f.read()

#text = """<s>L'oeuvre du talentueux {\{\{photographe\,\.w\+type="noun"\}\,\.roleName\+type="office"\+grfroleName\},.preInclude+grfpersContextePersonne} {\{Tristan\,\.forename\+grftagPrenom\} \{Garil\,\.surname\+grftagNomFamille\},.persName+grfpersPrenomNom} s'apprête à être exposée dans la {\{galerie\,\.w\+type="noun"\+kind\},.org+type="local"+quaero="ent"+grforgProximite} {\{\{Delandin\,\.surname\}\,\.persName\+grfpersGenerique\},.orgName+grforgSeule}.</s> <s>Mais la {\{directrice\,\.w\+type="noun"\},.roleName+type="office"+grfroleName} et fondatrice du lieu, {\{Estelle\,\.forename\+grftagPrenom\} \{Delandin\,\.surname\+grftagNomFamille\},.persName+grfpersPrenomNom}, est assassinée.</s>"""
new_text = parser.convert(text=text)

with open("corpus_result.txt", "w", encoding="utf-8") as f:
    f.write(new_text)

KeyboardInterrupt: 

In [35]:
from tools import new_ner
importlib.reload(new_ner)

Ner = new_ner.NER(
    dfs=[c_df, s_df]
)

Ner.merge()
ner = Ner.apply_correction()

In [ ]:
ner.to_excel("ner3.xlsx", index=False)